In [1]:
import sqlite3
import pandas as pd
import numpy as np
import haversine as hs

In [2]:
conn = sqlite3.connect('/home/vikash/scb-test/fsdata.db/fsdata.db')

In [3]:
venue_ratings=pd.read_sql_query("select user_id,venue_id,latitude,longitude,created_at from checkins", conn)

In [4]:
venue_ratings

,user_id,venue_id,latitude,longitude,created_at
0,539270,1206,41.878114,-87.629798,2011-12-08 05:08:42
1,1330941,1206,0.000000,0.000000,2011-12-08 04:32:19
2,1330942,1206,0.000000,0.000000,2011-12-08 04:29:38
3,282798,1206,41.878114,-87.629798,2011-12-08 04:26:06
4,376793,1206,41.878114,-87.629798,2011-12-08 04:17:50
...,...,...,...,...,...
1021961,244608,11138,0.000000,0.000000,2012-04-23 01:47:05
1021962,2153502,783,0.000000,0.000000,2012-04-23 01:42:42
1021963,592192,82919,40.239812,-76.919974,2012-04-22 23:26:48
1021964,494946,68691,32.912624,-96.638883,2012-04-23 00:36:33


In [41]:
# Top 10 users with most checkins
checkins=venue_ratings.groupby(['user_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)

In [42]:
list(checkins[:10].user_id)

[1348362,
 1900906,
 1326476,
 386648,
 1365850,
 467043,
 651415,
 304865,
 8622,
 439413]

In [5]:
#it seems form of the chekins don't have lat,lng values so lets uses the venues lat,lng values

In [6]:
venue_ratings=venue_ratings.drop(['latitude', 'longitude'], axis=1)

In [7]:
venues=pd.read_sql_query("select id,latitude,longitude from venues", conn)

In [8]:
merged=venue_ratings.merge(venues, left_on='venue_id', right_on='id')

In [9]:
merged=merged.drop(['id'], axis=1)

In [10]:
users=pd.read_sql_query("select id,latitude,longitude from users", conn)

In [11]:
merged=merged.merge(users, left_on='user_id', right_on='id')

In [12]:
merged=merged.drop(['id'], axis=1)

In [13]:
merged['distance'] = merged.apply(lambda x: hs.haversine((x.latitude_x,x.longitude_x),(x.latitude_y,x.longitude_y)), axis=1)

In [15]:
merged=merged.drop(['latitude_x', 'longitude_x','latitude_y', 'longitude_y'], axis=1)

In [24]:
merged

,user_id,venue_id,created_at,distance
0,539270,1206,2011-12-08 05:08:42,5.232317
1,539270,4424,2011-12-23 03:48:38,4.923552
2,539270,85492,2011-12-10 16:47:05,4.444228
3,539270,801995,2011-12-11 07:13:10,4.865394
4,1330941,1206,2011-12-08 04:32:19,1148.299470
...,...,...,...,...
1021961,2153065,71109,2012-04-22 23:21:19,11958.457381
1021962,2153171,733647,2012-04-22 22:52:20,11954.038685
1021963,1218328,792898,2012-04-22 23:26:48,14.577445
1021964,2153214,1094484,2012-04-22 22:42:14,12047.102301


In [26]:
merged=merged.sort_values(['user_id','venue_id','created_at'])

In [28]:
merged

,user_id,venue_id,created_at,distance
148480,1,51,2011-12-12 19:54:14,17.574260
148476,1,60,2012-04-11 22:21:26,2077.589448
148477,1,64,2012-01-29 22:56:11,28.470060
148478,1,7489,2012-01-29 17:06:02,872.146053
148479,1,28192,2012-04-13 01:06:20,2077.236568
...,...,...,...,...
759321,2153498,91385,2012-04-23 01:02:34,12581.506957
8705,2153499,783,2012-04-23 01:44:21,12576.703821
759323,2153500,91385,2012-04-22 23:10:54,12581.506957
755261,2153501,68691,2012-04-23 00:39:34,10517.277372


In [34]:
merged.created_at=pd.to_datetime(merged.created_at)

In [35]:
merged['days_since_last_visit']=merged.groupby(['user_id','venue_id'])['created_at'].diff().dt.days

In [44]:
merged[merged['user_id']==1900906]

,user_id,venue_id,created_at,distance,days_since_last_visit
216170,1900906,4700,2012-03-17 11:24:31,365.278428,NaN
216171,1900906,4700,2012-03-18 16:31:04,365.278428,1.0
216172,1900906,4700,2012-03-20 12:08:27,365.278428,1.0
216173,1900906,4700,2012-03-23 17:13:31,365.278428,3.0
216174,1900906,4700,2012-04-01 11:12:28,365.278428,8.0
216175,1900906,4700,2012-04-02 11:54:23,365.278428,1.0
216176,1900906,4700,2012-04-22 15:39:50,365.278428,20.0
216145,1900906,5222,2012-01-28 15:36:18,662.889841,NaN
216146,1900906,5222,2012-01-29 13:16:23,662.889841,0.0
216147,1900906,5222,2012-01-30 16:23:23,662.889841,1.0
